
How to use Bril with real code 

1. Bril is very simple, very regular, ir. 
2. Bril can be extended easily.
1. Bril has lots of tools and examples.

## Lets look at a bril program.

Bril is written in JSON format.  Almost all programming languages have a way to read json.


In [5]:
import json
import subprocess
import os 

with open("images/add.json", "r") as f:
    bril_program = json.load(f)
    
print(json.dumps(bril_program, indent=2))


{
  "functions": [
    {
      "name": "main",
      "instrs": [
        {
          "op": "const",
          "type": "int",
          "dest": "v0",
          "value": 1
        },
        {
          "op": "const",
          "type": "int",
          "dest": "v1",
          "value": 2
        },
        {
          "op": "add",
          "type": "int",
          "dest": "v2",
          "args": [
            "v0",
            "v1"
          ]
        },
        {
          "op": "print",
          "args": [
            "v2"
          ]
        }
      ],
      "args": []
    }
  ]
}


---

I reformatted this to make it simpler  (this is the real Bril format, not the text format)

```
{
  "functions": [
    {
      "name": "main",
      "instrs": [
        { "op": "const", "type": "int", "dest": "v0", "value": 1 },
        { "op": "const", "type": "int", "dest": "v1", "value": 2 },
        { "op": "add", "type": "int", "dest": "v2",
          "args": ["v0", "v1"] },
        { "op": "print", "args": ["v2"] }
      ],
      "args": []
    }
  ]
}
```



## getting started  

links:

1. [Language specification](https://capra.cs.cornell.edu/bril/)

2. [github site ](https://github.com/sampsyo/bril.git)


# clone the bril repo on a linux or wsl machine 

git clone https://github.com/sampsyo/bril.git



---

A lot of bril tools were written by students who had different ideas on what language to use. So you need several versions of package managers 

---

[deno](https://deno.com/) is the runtime  for typescript/javascript

curl -fsSL https://deno.land/install.sh | sh

on my ubuntu machine   'sudo snap install deno' also worked 

you may need to add $HOME/.deno/bin to your $PATH.



---

install the bril interpreter, and the typescript to bril compiler 

```
cd bril
deno install brili.ts 
deno install --allow-env --allow-read ts2bril.ts





```

In [26]:
## running the interpreter 


!brili <images/add.json

!brili -p <images/add.json

# the -p flag turns on  profiling 

3
3
total_dyn_inst: 4


## tools that translate between bril as json and bril as text are in python and get installed with flit

~~~
pip install --user flit
cd bril-txt
flit install --symlink --user
~~~

the text form is more readable

we have programs ***bril2txt*** and ***bril2json*** that make it easy to convert. Keep in mind that the json format is Bril and thats where you will do all the work. 


In [14]:
!bril2txt < images/add.json

@main {
  v0: int = const 1;
  v1: int = const 2;
  v2: int = add v0 v1;
  print v2;
}


0

In [29]:
# you can connect tools via pipes 
# !bril2txt < images/add.json | bril2json 

In [30]:
!cat images/add.json
!bril2txt < images/add.json | bril2jsonS

{
  "functions": [
    {
      "name": "main",
      "instrs": [
        { "op": "const", "type": "int", "dest": "v0", "value": 1 },
        { "op": "const", "type": "int", "dest": "v1", "value": 2 },
        { "op": "add", "type": "int", "dest": "v2",
          "args": ["v0", "v1"] },
        { "op": "print", "args": ["v2"] }
      ],
      "args": []
    }
  ]
}
{
  "functions": [
    {
      "instrs": [
        {
          "dest": "v0",
          "op": "const",
          "type": "int",
          "value": 1
        },
        {
          "dest": "v1",
          "op": "const",
          "type": "int",
          "value": 2
        },
        {
          "args": [
            "v0",
            "v1"
          ],
          "dest": "v2",
          "op": "add",
          "type": "int"
        },
        {
          "args": [
            "v2"
          ],
          "op": "print"
        }
      ],
      "name": "main"
    }
  ]
}


---

 There is also a fast interpreter written in [Rust](https://www.rust-lang.org/tools/install)




---

## turnt 


Bril uses [turnt](https://github.com/cucapra/turnt) as a test tool

pip install --user turnt

Bril tools were mostly student projects,  as you think about your projects, you might consider adding a new tool.
you can setup Bril on your local linux (can be wsl) machine by cloning the bril github and installing all the tools




## Gen CFG 

Lets write a sample program - that generate  the cfg 

. . .

I'll do this in two steps

1. find all the basic blocks
2. add all the cfg edges 


You can also do this in a single step, adding cfg edges as soon as you reach the successor node.


## basic blocks from a list of instructions-  
keep adding instructions till we get to a terminator or a label
(do we add labels?)

. . . 

```
in: list of instrs 
out: list of lists of instrs 

blocks = []
curr_block = []
for each instr in list 
   if the instruction is not a label put it on curr_block
   if instr is a label or terminator 
      put curr_block on blocks
      curr_block = []

if curr_block is not empty add it to blocks
return blocks 
```

two labels in a row do not need another block


--- 

step 2 add edges 

```
find cfg: in is bril program in json 
for each function find the list of instructions 
 for each basic block
    get last_instr 
    if it is a terminator  br/jmp/ret 
      add edge from current block to successor  
    --- what do we want to do with call? 
    else it is a fall through
       add edge to next block
```

. . . 

we need a map (block_map) label->block so we can add edges for blocks that end in br/jmp - can build this while getting the blocks or we can put the label as 
the first instruction

how do we get fall through? 

what about a return

if every block ends with a terminator, and every block has a label, then no fall through case 
what happens if try to delete the terminator (because the block never executes)


## code 

I'll use a python data structure called OrderedDict, when you iterate over the items in a ordered dict,  they come back in the order that they were installed.


## playing around with hacking- I'll use a generator 


```{python code-line-numbers="7,9| 8,10"}

#Instructions that terminate a basic block.
TERMINATORS = 'br', 'jmp', 'ret'


def form_blocks(instrs):
    """Given a list of Bril instructions, generate a sequence of
    instruction lists representing the basic blocks in the program.

    Every instruction in `instr` will show up in exactly one block. Jump
    and branch instructions may only appear at the end of a block, and
    control can transfer only to the top of a basic block---so labels
    can only appear at the *start* of a basic block. Basic blocks may
    not be empty.
    """

    # Start with an empty block.
    cur_block = []

    for instr in instrs:
        if 'op' in instr:  # It's an instruction.
            # Add the instruction to the currently-being-formed block.
            cur_block.append(instr)

            # If this is a terminator (branching instruction), it's the
            # last instruction in the block. Finish this block and
            # start a new one.
            if instr['op'] in TERMINATORS:
                yield cur_block
                cur_block = []

        else:  # It's a label.
            # End the block here (if it contains anything).
            if cur_block:
                yield cur_block

            # Start a new block with the label.
            cur_block = [instr]

    # Produce the final block, if any.
    if cur_block:
        yield cur_block
````

In [11]:
# as a test, lets print out the blocks 

def print_blocks(bril):
    """Given a Bril program, print out its basic blocks.
    """


    func = bril['functions'][0]  # We only process one function.
    for block in form_blocks(func['instrs']):
        # Mark the block.
        leader = block[0]
        if 'label' in leader:
            print( f"block {leader['label']}")
            block = block[1:]  # Hide the label, for concision.
        else:
            print('anonymous block:')

        # Print the instructions.
        for instr in block:
            print(instr)


print_blocks(bril_program)

anonymous block:
{'op': 'const', 'type': 'int', 'dest': 'v0', 'value': 1}
{'op': 'const', 'type': 'int', 'dest': 'v1', 'value': 2}
{'op': 'add', 'type': 'int', 'dest': 'v2', 'args': ['v0', 'v1']}
{'op': 'print', 'args': ['v2']}


In [28]:
with open("images/jmp.bril", 'r') as f:
          test2 = f.read()

print(test2)


result = subprocess.check_output('bril2json <images/jmp.bril', shell=True)

test2json = json.loads(result)
print(test2json)

FileNotFoundError: [Errno 2] No such file or directory: 'images/jmp.bril'

In [ ]:
print_blocks(test2json['functions'][0])

In [ ]:
# now for the map 
from collections import OrderedDict


def block_map(blocks):
    """Given a sequence of basic blocks, which are lists of instructions,
    produce a `OrderedDict` mapping names to blocks.

    The name of the block comes from the label it starts with, if any.
    Anonymous blocks, which don't start with a label, get an
    automatically generated name. Blocks in the mapping have their
    labels removed.
    """
    by_name = OrderedDict()

    for block in blocks:
        # Generate a name for the block.
        if 'label' in block[0]:
            # The block has a label. Remove the label but use it for the
            # block's name.
            name = block[0]['label']
            block = block[1:]
        else:
            # Make up a new name for this anonymous block.
            name = f'gen_bk_{len(by_name)}'

        # Add the block to the mapping.
        by_name[name] = block

    return by_name


blks = form_blocks(test2json['functions'][0]['instrs'])
od = block_map(blks)
for (name, instrs) in od.items():
    print (name, instrs)

# finally the cfg

~~~
out cfg = {} map label -> list of labels the successors of the block

for i , block in enumerate(blocks)  # blocks is a ordereddict 
  last = block[i]  # last instruction
  if last is jmp:
     cfg[block_name] = jmp.dest
  elif last is br:
    cfg[block.name] = [ last.if_label, last.else_label]
  else
     # fall through
    cfg[block_name = blocks[i+1].name  ## special case for last block
~~~

In [ ]:
def get_cfg(ordered_blocks):
    cfg = {}

    labels = list(ordered_blocks.keys())

    for i, (block_name, block) in enumerate(ordered_blocks.items()):
        last = block[-1]
        op = last['op']

        if op == 'jmp':
            cfg[block_name] = last['labels']
        elif op == 'br':
            cfg[block_name] = last['labels']
        else:
            if i+1 < len(labels):  # last block does not fall through
                cfg[block_name] = [labels[i+1]]
    return cfg


blks = form_blocks(test2json['functions'][0]['instrs'])
od = block_map(blks)
cfg = get_cfg(od)

print(cfg)